In [52]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [53]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [54]:
dfa = read_data()

In [55]:
dfa = dfa.fillna("")

In [56]:
dfa.loc[:, "state"] = dfa.state.str.lower().str.strip()
dfa.state.unique()

array(['wy', 'me', 'nm', 'nh', 'ok', 'nc', 'nj', 'ca', 'ga', 'vt', 'ak',
       'id', 'ri', 'tx', 'pa', 'ne', 'ny', 'in', 'mt', 'or', 'wa', 'ma',
       'al', 'va', 'il', 'nv', 'wi', 'fl', 'mi', 'oh', 'co', 'ct', 'sc',
       'la', 'ia', 'ar', 'az', 'tn', 'ut', 'ks', 'ky', 'mn', 'wa.', 'mo',
       'md', 'hi', 'wv', 'ms', 'dc', 'de', 'sd', 'nd', 'ae', '', 'canada',
       'grand island', 'sacramento', 'thailand', 'ne.', 'fl  (az)', 'ma.',
       'la.', 'n.y.', 'pa.', 'iowa', 'bc canada', '??', 'austraila pto',
       'ap', 'lv', 'jacksonville', 'bc', 'oslo', 'alberta, canada',
       'western australia', 'rl', 'carolina beach', '--', 'cn', 'pr',
       'md�', 'florida', '9000', 'stewartstown', 'roosevelt',
       'ca  nonono', 'e7n2v2', 'st croix', 'hamden', 'ontario',
       '607ws/dow, unit: 15173, apo, ap 96205, united states',
       'oh  start', 'mo  end', 'ny11580', '8724', 'ny11766', 'nc28773',
       'sterling heights'], dtype=object)

In [57]:
dfa = dfa[dfa.state.isin(["ct"])]
dfa.loc[:, "notes"] = dfa.notes.str.lower().str.strip()

In [58]:
dfa.loc[:, ["first_name", "last_name"]].iloc[:50]

,first_name,last_name
52,damon,bean
71,paul,cortesi
138,michael,dallas
385,damon,bean
416,kent,johnson
440,clifford,cuseo
649,jason,hartnagle
651,richard,kopec
688,william,chase jr
698,thomas,oravits jr


In [91]:
dfa.email.iloc[475:500]

38228    tkbowhunter103@gmail.com
Name: email, dtype: object

In [95]:
search_1 = dfa[dfa.notes.str.contains("law enforcement")]
search_1.notes.unique()

array(['military skills/tactics, law enforcement skills/tactics/knowledge, communications: general class amateur radio license, ,'],
      dtype=object)

In [92]:
search_2 = dfa[dfa.notes.str.contains("police")]
search_2.notes.unique()

array(['many of my patients are police officers, troopers, fbi agents, cia and in the military. i have the ability to spread the word of this organization to my patients. i showed 2 troopers this week the web link.',
       'served as an nbc officer, supply officer and training officer. instructor for command and general staff college for the last ten years of service. police officer in hartford ct for 10 years.'],
      dtype=object)

In [62]:
dfs = [search_1, search_2]
dfb = pd.concat(dfs, axis=0)
dfb

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name
31210,33844,Annual,No,Lebanon,ct,cpdt1@aol.com,42428.6841,"military skills/tactics, law enforcement skill...",dana,,petkaitis
10588,8592,Annual,No,Southbury,ct,davidmader@earthlink.net,,"many of my patients are police officers, troop...",dr,,. david mader
18262,21571,Annual,No,Winsted,ct,nhunt0656@charter.net,4/10/2013,"served as an nbc officer, supply officer and t...",neil,v,hunt


In [63]:
# def read_misconduct_data():
#     df = pd.read_csv("../data/HI/allegation_honolulu_pd_2021.csv")
#     return df

In [64]:
# dfc = read_misconduct_data()

In [65]:

def match_ok_names_to_personnel(ok, agency):
    dfa = ok[["first_name", "last_name"]]
    dfa.loc[:, "first_name"] = dfa.first_name.str.lower().str.strip()
    dfa.loc[:, "last_name"] = dfa.last_name.str.lower().str.strip()


    dfb = agency[["first_name", "last_name"]]
    dfb.loc[:, "first_name"] = dfb.first_name.str.lower().str.strip()
    dfb.loc[:, "last_name"] = dfb.last_name.str.lower().str.strip()

    matcher = ThresholdMatcher(
        ColumnsIndex([]),
        {
            "first_name": JaroWinklerSimilarity(),
            "last_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/HI/oathkeeper_names_v_HI_misconduct_data.xlsx"), decision
    )
    return matcher


In [66]:
# match = match_ok_names_to_personnel(dfb, dfc)

In [67]:
# def read_matches():
#     df = pd.read_excel("../data/HI/oathkeeper_names_v_HI_misconduct_data.xlsx")
#     return df 

In [68]:
# matches = read_matches()

In [69]:
# matches